In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame.from_dict({
    'month': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun'],
    'eggs': [33, 22, 13, 28, 43, 55],
    'salt': [12, 32,  9,  3,  23,  0], 
    'spam': [17, 31, 72, 20, 52, 55]
})
df.set_index('month', inplace=True)
df

# 基础

### Indexing value

In [ ]:
# 使用中括号
df["eggs"]['May']

# 使用列名
df.eggs['May']

# 使用 loc
df.loc['May', 'eggs']
df.loc['May']['eggs']

# 使用 iloc
df.iloc[4, 0]

### Slicing DataFrames

In [ ]:
# eggs 列中的某些值
df['eggs'][1:3]

# eggs 列中的某个值
df['eggs'][1]

# 某些列中的所有行
df.loc[:, 'eggs':'salt']

# 某些行中所有列
df.loc['Jan':'Feb', :]

# 某些行中某些列
df.loc['Jan':'Mar', 'eggs':'salt']
df.loc['Jan':'Mar', ['eggs', 'salt']]  # 使用列表删选行

# 使用 iloc 选择
df.iloc[2:5, 1:]

# df['eggs']  -> pandas.core.series.Series
# df[['eggs]] -> pandas.core.frame.DataFrame
pass

### Filtering

In [ ]:
df.loc[:, df.all()]           # 所有非0值
df.loc[:, df.any()]           # 0值行
df.loc[:, df.isnull().any()]  # 任何 na 行
df.loc[:, df.notnull().any()] # 任何 na 行
df.dropna(how='any')
pass

### Manipulating

In [ ]:
df.eggs[df.salt > 55] += 5   # 加
df.floordiv(12)              # 类似 dataframe // other，支持fill_value
pass

# 索引

### About DataFrame Index

In [ ]:
# 创建 Series
prices = [3, 2, 1.5, 4, 4.5]
shares = pd.Series(prices)

# 创建Series，从数组指定索引值
days = ['Mon', 'Tue', 'Wed', 'Thur', 'Fri']
shares = pd.Series(prices, index=days)

In [ ]:
# 可使用下标访问索引值，也可以切片
shares.index[0]

# 给索引起名
shares.name = "Share"

# 给索引赋值会出错

In [ ]:
# 多重索引访问索引名
# df.index.name  -> None
# df.index.names -> ['a', 'b']

# 按索引排序
df.sort_index()

# 指定索引的顺序进行排序
df.reindex(['Jan', 'Apr'])

# 按照另外一个dataframe index 的顺序排序
df2 = pd.DataFrame()
df.reindex(df2.index)

# 多重索引选择，使用元组
df.loc[('idx1','idx2'), 'col_name']

# 只选择单个索引，会返回该索引内部得值（包括子索引）
df.loc['idx1']

# 使用 slice 函数，对第一层不设条件，返回第二层 ab 结果
df.loc[slice(None), slice('a','b')]

### Melt

- Melt：适用于列中存在值，需要整理到行中的情况
- Pivoting: 与Melt 相反，将1列中唯一的值放到不同的列上

### Pivot

- Pivot 根据列值 Reshape 数据，有重复值会出错
- PivotTable 根据列值 Reshape 数据，有重复值可指定 aggregate 函数(sum, count, etc.)
- 都可以指定多个 index 或者 columns

### Stack 和 Unstack

- UnStack 将行上的多 index 中的某一个放到 column 上，需要指定参数 level= '' ，即index 中的某个值；传给 level 的也可以是一个数字比如 0,1 等，取决于 index 的层数
- Stack 将 column 上的 index 放到行上
- 多索引中，可使用 swaplevel 方式交换两个索引的顺序

### Group by

Groupby 一般有几个步骤：

- 根据某些列按其值分割成不同的组
- 对每个组，指定内部的某些列进行操作
- 将这些操作后的结果组合起来

Groupby 有一些特殊操作方便我们使用：
- 第一个分割组的操作，可以传入自定义的 Series, 不限定于列名
- groupby 的时候，使用 category 类型的数据会加快速度
- 组合组内部的操作，一般使用 mean, sum, max 等，但也可以一次指定多个函数，如 `df.groupby(['a','b']).agg('a':'sum')`, 也可以指定自定义的函数

Groupby 可以和 transformation 结合使用，看下面的例子：

```python
 def zscore(series):
     "对 Series 中的值，每一个都减去其平均值，再除以标准差"
    return (series - series.mean()) / series.std()
```

使用的时候，直接传入 dataframe 的某列：

```python
zscore(df['mpg']).head()
```

所以，我们也可以接 group 去使用：

```python
df.groupby('yr')['mpg'].transform(zscore).head()
```

进一步的，可以把一些操作组合起来放到一个函数:

```python
def zscore_with_year_and_name(group):
    """针对每一个 group，只取其中的某些列（mpg, year, name）做一些操作并组合成新的 dataframe
       对于操作：mpg，算 zscore; yr 重命名为 year; name 不操作.
    """
    df = pd.DataFrame({
            'mpg': zscore(group['mpg']),
            'year': group['yr'],
            'name': group['name']
        })
    return df

df.groupby('yr').apply(zscore_with_year_and_name).head()
```

有时候你分组操作后，需要再进行 filter 操作，这需要拿到 groupby 之后的对象进行遍历操作。

首先我们来看看 groupby 操作后得到的对象：

```python
splitting = auto.groupby('yr')

type(splitting)
# pandas.core.groupby.DataFrameGroupBy

type(splitting.groups)
# dict

print(splitting.groups.keys())
# dict_keys([70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82])
```

你可以对 group 对象进行遍历:

```python
for groupname, group in splitting:
   avg = group['mpg'].mean()
   print(group_name, avg)
```

对于一个 group 对象，你也可以对其使用 loc 操作，所以在内部我们可以进行 filter:

```python
# 对 group filter，只找出 name 包含 chevrolet 的组，取出其 mpg 列求 mean
# 返回的是单个值
group.loc[group['name'].str.contains('chevrolet'), 'mpg'].mean()
```

```python
# 结合操作，得到一个 Series
chevy_means = {
    year: group.loc[group['name'].str.contains('chevrolet'),'mpg'].mean()
    for year, group in splitting
}
pd.Series(chevy_means)
```

一组条件，也可以当作 groupby 的列，如:


```python
# chevy 一组 True, False，其值反映了df name列中包含 chevrolet 的行
chevy = df['name'].str.contains('chevrolet')

# 将 chevy 作为 groupby 条件
df.groupby(['yr', chevy])['mpg'].mean()
```

### Concat

- Concat 时可以指定 index，如 `pd.concat([rain2013, rain2014], keys=[2013, 2014], axis=0)`，如果两个 df 本来就有 index，则会产生多 index
- index 也可以在 column 上，如 `pd.concat([rain2013, rain2014], keys=[2013, 2014], axis='columns')`
- 可以将两个数组左右拼接到一起，可以使用 `np.hstack([B, A])` 或者 `np.concatenate([B, A], axis=1)`

### Merge

- merge 时，指定 suffixis 参数来区分来自不同 dataframe 的列
- 对于特殊的 merge，可以了解 `pandas.merge_asof` 和 `pandas.merge_ordered` 方法